In [1]:
!pip install mediapipe opencv-python

In [2]:
import cv2
import mediapipe as mp
import numpy as np

mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

STATUS_GO = True
stage = False
counter = 0

FLAG_DO = False
FLAG_GO = False
FLAG_STOP = False

Test = False

In [10]:
def calculate_angle(a,b,c):
    a = np.array(a) # First
    b = np.array(b) # Mid
    c = np.array(c) # End
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    
    if angle >180.0:
        angle = 360-angle
        
    return angle

def replace_trigger(flag):
    if flag:
        return "O"
    
    else:
        return "X"

def replace_status(status):
    if status:
        return 'GO'
    
    else:
        return 'STOP'

# They occur errors, so These functions are not used. but, main function use these logic
'''
def flag_do(l_elbow, l_wrist, l_shoulder):
    l_elbow = np.array[l_elbow]
    l_wrist = np.array[l_wrist]
    l_shoulder = np.array[l_shoulder]
    
    print(l_elbow[1], l_wrist[1])
    
    if (l_elbow[1] > l_wrist[1]):
        FLAG_DO = True   
    
    if (l_elbow[1] > l_wrist[1]) and (l_shoulder[0] > l_wrist[0]):
        FLAG_DO = True
    
    else:
        FLAG_DO = False
    
    
def flag_stop(l_elbow, r_wrist):
    l_elbow = np.array[l_elbow]
    r_wrist = np.array[r_wrist]
    
    if stage:
        if (l_elbow[0] < r_wrist[0]):
            stage = True
            counter += 1
            
            if (counter >= 2):
                counter = 0
                return True
            else:
                return False
        return False
        
    else:
        if (l_elbow[0] > r_wrist[0]):
            stage = False
            
        return False
    
    
def flag_go(r_wrist, r_elbow, r_shoulder):
    if stage:
        if (calculate_angle(r_wrist, r_elbow, r_shoulder) < 100):
            stage = True
            counter += 1
            
            if (counter >= 2):
                counter = 0
                return True
            else:
                return False
        return False
        
    else:
        if (calculate_angle(r_wrist, r_elbow, r_shoulder) > 120):
            stage = False
            
        return False
'''


'\ndef flag_do(l_elbow, l_wrist, l_shoulder):\n    l_elbow = np.array[l_elbow]\n    l_wrist = np.array[l_wrist]\n    l_shoulder = np.array[l_shoulder]\n    \n    print(l_elbow[1], l_wrist[1])\n    \n    if (l_elbow[1] > l_wrist[1]):\n        FLAG_DO = True   \n    \n    if (l_elbow[1] > l_wrist[1]) and (l_shoulder[0] > l_wrist[0]):\n        FLAG_DO = True\n    \n    else:\n        FLAG_DO = False\n    \n    \ndef flag_stop(l_elbow, r_wrist):\n    l_elbow = np.array[l_elbow]\n    r_wrist = np.array[r_wrist]\n    \n    if stage:\n        if (l_elbow[0] < r_wrist[0]):\n            stage = True\n            counter += 1\n            \n            if (counter >= 2):\n                counter = 0\n                return True\n            else:\n                return False\n        return False\n        \n    else:\n        if (l_elbow[0] > r_wrist[0]):\n            stage = False\n            \n        return False\n    \n    \ndef flag_go(r_wrist, r_elbow, r_shoulder):\n    if stage:\n      

In [12]:
cap = cv2.VideoCapture(0)

## Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
      
        # Make detection
        results = pose.process(image)
    
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark
            
            l_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            l_elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            l_wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
            
            
            #flag_do (l_elbow, l_wrist, l_shoulder, r_shoulder)
            if (l_elbow[1] > l_wrist[1]) and (l_shoulder[0] > l_wrist[0]):
                FLAG_DO = True
    
            else:
                FLAG_DO = False
                counter = 0

            
            if FLAG_DO:
                
                # seperate segments for unnecessary power
                r_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
                r_elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
                r_wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
                
                # flag_stop
                if STATUS_GO and (not stage):
                    if (l_elbow[1] < r_wrist[1]) and (r_shoulder[0] < r_wrist[0]):
                        stage = True
                        counter += 1
        
                        if (counter >= 3):
                            STATUS_GO = False
                            counter = 0
                            stage = False
    
                elif STATUS_GO and stage:
                    if (l_elbow[1] > r_wrist[1]):
                        stage = False
                
                # flag_go
                elif (not STATUS_GO) and (not stage):
                    if (r_wrist[1] < r_shoulder[1]) and (calculate_angle(r_wrist, r_elbow, r_shoulder) < 100):
                        stage = True
                        counter += 1
            
                        # because of purpose, '+1' is needed
                        if (counter >= 4):
                            STATUS_GO = True
                            counter = 0
                            stage = False
                    
                elif (not STATUS_GO) and stage:
                    if (r_wrist[1] < r_shoulder[1]) and (calculate_angle(r_wrist, r_elbow, r_shoulder) > 110):
                        stage = False
                
            
        except:
            pass
        
        
        # Setup status box
        cv2.rectangle(image, (0,0), (225,73), (245,117,16), -1)
        
        # Trigger data
        cv2.putText(image, 'Trigger', (5,12), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, replace_trigger(FLAG_DO), 
                    (10,60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
        
        # Status data
        cv2.putText(image, 'Status', (65,12), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, replace_status(STATUS_GO), 
                    (60,60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)      
        
        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                 )          
        
        # Show image
        cv2.imshow('Mediapipe Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()